In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
#import sys.path.append("/home/eze/Escritorio/Exactas/04_métodosNuméricos/07_TP/del_git")
import moduloALC
import alc

In [2]:
Xt, Yt, Xv, Yv = alc.cargarDataset("./cats_and_dogs")

In [18]:
Xt

array([[ 0.01321056,  0.39967388,  0.4646435 , ...,  0.02506035,
        -0.04273714,  0.15228784],
       [-0.08923571,  0.02555032,  0.18569547, ...,  0.40389472,
         0.22538967,  0.17212628],
       [ 0.02017952, -0.09116424, -0.09575794, ..., -0.13116124,
        -0.10845048,  0.34039333],
       ...,
       [-0.10537947, -0.03899113,  0.26660895, ...,  0.14557125,
         0.16828245,  0.03130165],
       [ 0.24019487,  0.04183483, -0.10228873, ...,  0.03360566,
        -0.02517227,  0.6137086 ],
       [ 0.08662421,  0.00934677,  0.22371952, ..., -0.06815872,
        -0.12536943,  0.47931114]], shape=(1536, 2000), dtype=float32)

In [19]:
Yt

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]], shape=(2, 2000))

In [20]:
print(Xt[0:5,0:5])


[[ 0.01321056  0.39967388  0.4646435  -0.04907167 -0.03964727]
 [-0.08923571  0.02555032  0.18569547 -0.04128886  0.41182464]
 [ 0.02017952 -0.09116424 -0.09575794  0.01279821  0.3294663 ]
 [ 0.24558485  0.26327386  0.25831386  0.74906355  0.31219542]
 [ 0.13532521  0.08871768  0.5993278   0.33613265  0.16824257]]


### Descomposiciones aparte (después)

In [21]:
#Cholesky

In [4]:
# W = alc.algoritmo1(Xt[:,990:1010], Yt[:,990:1010])
W = alc.algoritmo3(Xt[:,990:1010], Yt[:,990:1010])


TypeError: cannot unpack non-iterable NoneType object

In [4]:
W@Xv[:, 490:510]

array([[ 1.01085971,  1.49620514,  0.51487335,  0.3621459 ,  0.53207344,
         0.40927935,  0.87886204,  0.59791893,  1.04003769,  0.71261974,
         0.89991235,  0.69188537,  0.58445698,  1.03213546,  0.46465441,
         0.67185039,  0.49917932,  0.76556937,  0.57512478,  0.6864771 ],
       [ 0.1049696 , -0.12367449,  0.39592837,  0.37468765,  0.22630189,
         0.72234097,  0.09164559,  0.14746492,  0.41874707,  0.15889907,
         0.23144099,  0.36462437,  0.2087051 ,  0.467239  ,  0.67988916,
         0.6906388 ,  0.36695054,  0.13419724,  0.58093649,  0.31899865]])

In [5]:
Yv[:, 490:510]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.]])